# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire
tol = 1e-12
is_converged = DFTK.ScfConvergenceDensity(tol);

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846826292999                   -0.70    4.8
  2   -7.852314911416       -2.26       -1.53    1.0
  3   -7.852646217768       -3.48       -2.52    3.2
  4   -7.852646677836       -6.34       -3.37    2.0
  5   -7.852646685855       -8.10       -4.72    1.2
  6   -7.852646686723       -9.06       -5.22    3.5
  7   -7.852646686730      -11.19       -6.22    1.5
  8   -7.852646686730      -12.69       -7.53    2.2
  9   -7.852646686730      -14.75       -7.69    2.2
 10   -7.852646686730   +    -Inf       -8.68    2.0
 11   -7.852646686730   +    -Inf       -9.43    2.0
 12   -7.852646686730   +  -14.45      -10.38    2.2
 13   -7.852646686730      -14.75       -9.59    1.0
 14   -7.852646686730      -14.75       -9.35    1.2
 15   -7.852646686730   +    -Inf       -9.76    1.0
 16   -7.852646686730   +  -14.45       -9.68    1.0
 17   -7.852646686730      -14.21      -10.75 

## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag
---   ---------------   ---------   ---------   ----   ----
  1   -7.846749508361                   -0.70           4.8
  2   -7.852550497239       -2.24       -1.63   0.80    2.0
  3   -7.852638082853       -4.06       -2.74   0.80    1.0
  4   -7.852646444189       -5.08       -3.34   0.80    2.0
  5   -7.852646670131       -6.65       -4.17   0.80    1.2
  6   -7.852646686387       -7.79       -4.70   0.80    1.8
  7   -7.852646686712       -9.49       -5.52   0.80    1.2
  8   -7.852646686730      -10.75       -6.24   0.80    2.0
  9   -7.852646686730      -12.61       -7.69   0.80    1.8
 10   -7.852646686730      -13.80       -8.54   0.80    2.8
 11   -7.852646686730   +  -15.05       -9.04   0.80    2.0
 12   -7.852646686730      -15.05       -9.81   0.80    1.5
 13   -7.852646686730   +  -15.05       -9.96   0.80    2.2
 14   -7.852646686730      -15.05      -10.96   0.80    1.0
 15   -7.852646686730      -15.05      -

## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol);

Iter     Function value   Gradient norm 
     0     1.398023e+01     3.487827e+00
 * time: 0.5154929161071777
     1     1.284146e+00     1.875688e+00
 * time: 0.7912449836730957
     2    -1.973630e+00     1.837036e+00
 * time: 0.8193328380584717
     3    -3.975799e+00     1.600702e+00
 * time: 0.859889030456543
     4    -5.488606e+00     1.453389e+00
 * time: 0.9020228385925293
     5    -7.079678e+00     8.082662e-01
 * time: 0.9437038898468018
     6    -7.248951e+00     9.653748e-01
 * time: 0.9716529846191406
     7    -7.674305e+00     5.440029e-01
 * time: 0.9997508525848389
     8    -7.780056e+00     2.474160e-01
 * time: 1.0307269096374512
     9    -7.819809e+00     7.655316e-02
 * time: 1.0585570335388184
    10    -7.836319e+00     1.022983e-01
 * time: 1.0868239402770996
    11    -7.845444e+00     5.819173e-02
 * time: 1.115006923675537
    12    -7.850422e+00     2.541396e-02
 * time: 1.1474108695983887
    13    -7.851877e+00     1.906402e-02
 * time: 1.284502983093

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=1e-1);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846787272800                   -0.70    4.8
  2   -7.852296897408       -2.26       -1.53    1.0


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ, _ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation)
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)
---   ---------------   ---------   ---------
  1   -7.852646686695                   -2.54
  2   -7.852646686730      -10.46       -5.81
  3   -7.852646686730   +    -Inf      -12.34


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 5.007076931754657e-13
|ρ_newton - ρ_scfv| = 3.048497941303998e-13
|ρ_newton - ρ_dm|   = 2.428438753833466e-10
